In [ ]:
#rbf - euclidian distance??
#augumentation rotate zoom

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess MNIST data
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1)).astype('float32') / 255

In [ ]:
#some arctitecture
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from functools import partial

DefaultConvo2D = partial(layers.Conv2D, kernel_size=3, activation = 'relu', padding="SAME")

my_model = keras.models.Sequential([
    DefaultConvo2D(filters=64,kernel_size=7,input_shape=[28,28,1]),
    layers.MaxPooling2D(pool_size=2),
    DefaultConvo2D(filters=128),
    DefaultConvo2D(filters=128),
    layers.MaxPooling2D(pool_size=2),
    DefaultConvo2D(filters=256),
    DefaultConvo2D(filters=256),
    layers.MaxPooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(units=128,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(units=64,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(units=32,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(units=10, activation='softmax'),
])

my_model.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"])

hist=my_model.fit(train_images,train_labels,epochs=20)

In [ ]:
test_loss, test_acc = my_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

In [ ]:
#SE

import tensorflow as tf
from tensorflow import keras
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, activation = 'relu', padding="SAME")

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation=keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters,strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1,strides=strides),
                keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        z=inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z=inputs
        for layer in self.skip_layers:
            skip_z=layer(skip_z)
        return self.activation(z + skip_z)
        
model=keras.models.Sequential()
model.add(DefaultConv2D(64,kernel_size=7,strides=2,input_shape=[28,28,1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding='same'))
prev_filters=64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides=1 if filters==prev_filters else 2
    model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation='softmax'))

model.summary()